<a href="https://colab.research.google.com/github/AndriiFedorchuk/Andrii/blob/master/Main_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy 
import pandas as pd
from sklearn.model_selection import train_test_split


URL = 'https://raw.githubusercontent.com/AndriiFedorchuk/Andrii/master/main_fa.csv'
dataset = pd.read_csv(URL)
dataset.head(10)

,History,Psychology,Politics,Mathematics,Physics,Internet,PC,Economy Management,Biology,Chemistry,Reading,Geography,Foreign languages,Medicine,Law,Cars,Art exhibitions,Religion,Countryside and outdoors,Dancing,Musical instruments,Writing,Passive sport,Active sport,Gardening,Celebrities,Shopping,Science and technology,Theatre,Fun with friends,Adrenaline sports,Pets,Gender
0,1,5,1,3,3,5,3,5,3,3,3,3,5,3,1,1,1,1,5,3,3,2,1,5,5,1,4,4,2,5,4,4,1
1,1,3,4,5,2,4,4,5,1,1,4,4,5,1,2,2,2,1,1,1,1,1,1,1,1,2,3,3,2,4,2,5,1
2,1,2,1,5,2,4,2,4,1,1,5,2,5,2,3,1,5,5,5,5,5,5,5,2,1,1,4,2,5,5,5,5,1
3,4,4,5,4,1,3,1,2,3,3,5,4,4,2,5,1,5,4,1,1,1,3,1,1,1,2,4,3,1,2,1,1,1
4,3,2,3,2,2,2,2,2,3,3,5,2,3,3,2,3,1,4,4,1,3,1,3,1,4,3,3,3,2,4,2,1,1
5,5,3,4,2,3,4,4,1,4,4,3,3,4,4,3,5,2,2,5,1,5,1,5,4,2,1,2,3,1,3,3,2,0
6,3,3,1,1,1,2,1,3,5,5,3,3,4,5,3,4,1,1,4,3,2,1,5,3,3,1,3,4,3,5,1,5,1
7,5,2,3,1,1,5,4,1,2,2,2,3,4,1,2,1,1,2,2,1,1,1,4,5,1,3,3,2,2,4,2,5,0
8,3,2,1,1,1,1,1,1,3,1,5,1,1,1,1,1,1,2,4,1,2,1,4,1,1,5,2,1,5,4,1,1,1
9,3,2,3,3,1,5,1,4,2,1,4,4,5,1,1,1,4,4,4,5,3,1,4,4,1,2,4,3,5,5,2,2,1


In [0]:
x = dataset.iloc[:,:32].values
y = dataset.iloc[:,32].values

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.1)

In [22]:
# print info about a shape of arrays
print(x_train.shape, 'train samples for parameters')
print(y_train.shape, 'train samples for output - Gender')
print(x_test.shape, 'test samples  for parameters')
print(y_test.shape, 'test samples for output - Gender')

(900, 32) train samples for parameters
(900,) train samples for output - Gender
(100, 32) test samples  for parameters
(100,) test samples for output - Gender


In [23]:
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier


# train the model
#clf = GaussianNB()
#clf = LinearDiscriminantAnalysis()
clf = KNeighborsClassifier(5)
#clf =  MLPClassifier(alpha=0.05, max_iter=10000,solver='adam', random_state=1,
#                hidden_layer_sizes=[100, 100],early_stopping=True,
#                verbose=10, tol=0.00001)

print(clf.fit(x_train, y_train))
#print(clf.feature_importances_)  

# use the model to predict the labels of the test data
predicted = clf.predict(x_test)
expected = y_test


print("Score train = ",clf.score(x_train, y_train))
print("Score test  = ",clf.score(x_test, y_test))

from sklearn import metrics

print('Confusion matrix:')
print(metrics.confusion_matrix(expected, predicted))
plt.show()

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')
Score train =  0.89
Score test  =  0.83
Confusion matrix:
[[35  8]
 [ 9 48]]


In [24]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

# Neural network
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(32,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_46 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_47 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_48 (Dense)             (None, 2)                 66        
Total params: 3,234
Trainable params: 3,234
Non-trainable params: 0
_________________________________________________________________


In [25]:
batch_size = 100
num_classes = 2
epochs = 40

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))


Train on 900 samples, validate on 100 samples
Epoch 1/40
900/900 [==============================] - 0s 528us/step - loss: 0.6279 - acc: 0.6500 - val_loss: 0.5460 - val_acc: 0.7400
Epoch 2/40
900/900 [==============================] - 0s 23us/step - loss: 0.5241 - acc: 0.7578 - val_loss: 0.4788 - val_acc: 0.7800
Epoch 3/40
900/900 [==============================] - 0s 22us/step - loss: 0.4643 - acc: 0.8000 - val_loss: 0.4287 - val_acc: 0.7900
Epoch 4/40
900/900 [==============================] - 0s 20us/step - loss: 0.4146 - acc: 0.8167 - val_loss: 0.3889 - val_acc: 0.8200
Epoch 5/40
900/900 [==============================] - 0s 19us/step - loss: 0.3779 - acc: 0.8244 - val_loss: 0.3606 - val_acc: 0.8300
Epoch 6/40
900/900 [==============================] - 0s 25us/step - loss: 0.3538 - acc: 0.8489 - val_loss: 0.3554 - val_acc: 0.8300
Epoch 7/40
900/900 [==============================] - 0s 19us/step - loss: 0.3345 - acc: 0.8578 - val_loss: 0.3721 - val_acc: 0.8100
Epoch 8/40
900/900 [==